<a href="https://colab.research.google.com/github/Int-Group-44/flowers-102/blob/main/OxfordFlowers102CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

dataset, dataset_info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
dataset_info
test_set, training_set, validation_set = dataset['test'], dataset['train'], dataset['validation']

Found GPU at: /device:GPU:0


Importing TensorFlow 

In [2]:
num_classes = dataset_info.features['label'].num_classes
num_training_examples = 0
num_validation_examples = 0
for example in training_set:
    num_training_examples += 1
for example in validation_set:
    num_validation_examples += 1


In [3]:
IMAGE_RES = 224
def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    label = tf.one_hot(label, depth=num_classes)
    return image, label
BATCH_SIZE = 32
train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_set.shuffle(num_validation_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)


In [4]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Conv2D(128, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 102:
      activation = "softmax"
      units = 102
    else:
      activation = "sigmoid"
      units = 1

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


In [5]:
model = make_model(input_shape=(224,224) + (3,), num_classes=102)
#keras.utils.plot_model(model, show_shapes=True)

epochs = 50

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_batches,
    epochs=epochs,
    validation_data=validation_batches,
    batch_size=BATCH_SIZE,
    validation_batch_size=BATCH_SIZE,
    verbose=1
)

Epoch 1/50
32/32 [==============================] - 54s 1s/step - loss: 4.6522 - accuracy: 0.0441 - val_loss: 4.6213 - val_accuracy: 0.0186
Epoch 2/50
32/32 [==============================] - 21s 644ms/step - loss: 3.9565 - accuracy: 0.0804 - val_loss: 4.6207 - val_accuracy: 0.0137
Epoch 3/50
32/32 [==============================] - 20s 624ms/step - loss: 3.5487 - accuracy: 0.1422 - val_loss: 4.6458 - val_accuracy: 0.0098
Epoch 4/50
 5/32 [===>..........................] - ETA: 13s - loss: 3.3771 - accuracy: 0.1688

KeyboardInterrupt: ignored

In [116]:
tf.keras.backend.clear_session()

In [6]:
test_loss, test_acc = model.evaluate(train_batches, verbose=1, batch_size=BATCH_SIZE)
print("Test accuracy:", test_acc)

32/32 [==============================] - 6s 165ms/step - loss: 4.6537 - accuracy: 0.0098
Test accuracy: 0.009803921915590763


In [113]:
for features in train_batches.take(1):
  print(features)

(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
array([[[[2.20325626e-02, 3.77188362e-02, 3.37972678e-02],
         [2.20325626e-02, 3.77188362e-02, 3.37972678e-02],
         [2.20325626e-02, 3.77188362e-02, 3.37972678e-02],
         ...,
         [2.94296108e-02, 2.94296108e-02, 2.94296108e-02],
         [2.35294122e-02, 2.35294122e-02, 2.35294122e-02],
         [2.35294122e-02, 2.35294122e-02, 2.35294122e-02]],

        [[2.35294122e-02, 3.92156877e-02, 3.52941193e-02],
         [2.35294122e-02, 3.92156877e-02, 3.52941193e-02],
         [2.35294122e-02, 3.92156877e-02, 3.52941193e-02],
         ...,
         [2.63661519e-02, 2.63661519e-02, 2.63661519e-02],
         [2.35294122e-02, 2.35294122e-02, 2.35294122e-02],
         [2.35294122e-02, 2.35294122e-02, 2.35294122e-02]],

        [[2.74509806e-02, 4.31372561e-02, 3.92156877e-02],
         [2.74509806e-02, 4.31372561e-02, 3.92156877e-02],
         [2.74509806e-02, 4.31372561e-02, 3.92156877e-02],
         ...,
         